# Fix: Remove duplicates for multi-purpose indicators

## Load necessary libraries

In [1]:
import csv
import json
import urllib3  # allows to access a URL with python
import math
import os
import io
import collections
import copy
import re
import numpy as np
import pandas as pd
import xlsxwriter

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## User parameters

In [2]:
release = '2019.Q1.G.02' # Make sure to have the correct release here

dir_path = os.path.dirname(os.path.realpath('__file__'))
print("Path: " + dir_path)

wd_dir = r'../'
print('Main dir: ' + wd_dir)

Path: C:\Users\L.GonzalezMorales\Documents\GitHub\FIS4SDGs\unsd\data
Main dir: ../


## Utilities

#### Convert string to camelCase

In [3]:
def camelCase(st):
    """
    https://stackoverflow.com/questions/8347048/camelcase-every-string-any-standard-library
    
    """
    output = ''.join(x for x in st.title() if x.isalnum())
    return output[0].lower() + output[1:]

### Read list of excel files with data for each series

In [8]:
path = wd_dir + r'data/' + release + '/' 
print("path: " + path)
files = os.listdir(path)
files =  [x for x in files if x[-4:] == 'xlsx' and x[:4]=='Data']
files[0:3]


path: ../data/2019.Q1.G.02/


['Data_1.1.1_SI_POV_DAY1.xlsx',
 'Data_1.1.1_SI_POV_EMP1.xlsx',
 'Data_1.2.1_SI_POV_NAHC.xlsx']

### For a selected file:

#### 1)  Convert content of data file into a pandas DataFrame

In [10]:
f = 'Data_1.5.2_VC_DSR_LSGP.xlsx'

xlsx = pd.ExcelFile(path + f)
x = pd.read_excel(xlsx, 'Sheet1')
x.shape
x.columns
x.head(3)
x.tail(3)




(8060, 33)

Index(['goalCode', 'goalDesc', 'targetCode', 'targetDesc', 'indicatorCode',
       'indicatorDesc', 'indicatorTier', 'seriesCode', 'seriesDesc',
       'seriesRelease', 'timePeriod', 'geoAreaCode', 'geoAreaName', 'level',
       'parentCode', 'parentName', 'type', 'CountryProfile', 'ISO3CD',
       'UNMember', 'X', 'Y', 'value', 'valueType', 'timeDetail', 'source',
       'footnotes', 'natureCode', 'unitsCode', 'reportingTypeCode',
       'natureDesc', 'unitsDesc', 'reportingTypeDesc'],
      dtype='object')

,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,valueType,timeDetail,source,footnotes,natureCode,unitsCode,reportingTypeCode,natureDesc,unitsDesc,reportingTypeDesc
0,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,valueType,timeDetail,source,footnotes,natureCode,unitsCode,reportingTypeCode,natureDesc,unitsDesc,reportingTypeDesc
8057,11,"Make cities and human settlements inclusive, s...",11.5,"By 2030, significantly reduce the number of de...",11.5.2,Direct economic loss in relation to global GDP...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8058,11,"Make cities and human settlements inclusive, s...",11.5,"By 2030, significantly reduce the number of de...",11.5.2,Direct economic loss in relation to global GDP...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8059,11,"Make cities and human settlements inclusive, s...",11.5,"By 2030, significantly reduce the number of de...",11.5.2,Direct economic loss in relation to global GDP...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
